<a href="https://colab.research.google.com/github/Konstvv/DL_MRI_reconstriction/blob/main/MRI_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO list:
1. Find the dataset (High-quality MRI images)
* small dataset - https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
* large dataset - https://fastmri.med.nyu.edu/ (need to request the access)
* Something else?

2. Preprocess the data

 2.1 HQ images -> make them noisy

 * White noise - every pixel would get a SMALL random increment
 * Gaussian blur - https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html

 2.2 (maybe optional, depends on dataset size) Data augmentation
 * Rotation (saving the original size)
 * Zoom
 * Mirror

Training sample - pair of images (clean and corrupted images: label and sample)

3. Creation of the model

  * Supervised learning paradigm
  * Baseline model - CNN architecture (dims of input and output are the same)
  * Compare the output to the orginal image:
      * PSNR(Peak Signal-to-Noise Ratio)
      * SSIM
  * Overview of the models: https://arxiv.org/pdf/1912.13171.pdf
  * Brain MRI model: https://aapm.onlinelibrary.wiley.com/doi/epdf/10.1002/acm2.13758

  * General overview: https://towardsai.net/p/deep-learning/image-de-noising-using-deep-learning

4. Tuning and testing


In [ ]:
## Part 1
# Dataset import
# Upload it to Google Drive
# Fetch the data Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
filepath = '/content/drive/MyDrive'
for file in os.listdir(filepath):
  print(file)
  # Code that imports all the images
  # Two options of output:
  ## list with all the images (fast, a lot of RAM) -> preferable method
  ## create a separate folder with all the images + function to preprocess images in batches

In [ ]:
## Part 2
# Datset preprocessing

import numpy as np
import os
import matplotlib.pyplot as plt
import cv2

pic1 = plt.imread('cat.jpeg')
pic2 = plt.imread('parrot.jpg')
plt.imshow(pic1)
plt.figure()
plt.imshow(pic2)

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomContrast, RandomZoom

class NoiseLayer(tf.keras.layers.Layer):
  def __init__(self, noise_type='gauss'):
    super(NoiseLayer, self).__init__()
    self.noise_type = noise_type

  def build(self, input_shape):
    pass

  def call(self, inputs):
    if self.noise_type == "gauss":
        mean = 0
        var = 0.1
        gauss = np.random.normal(mean, var**0.5, inputs.shape)
        gauss = gauss.reshape(*inputs.shape)
        noisy = inputs + gauss
        return noisy
    elif self.noise_type == "poisson":
        peak = 5
        noisy = np.random.poisson(inputs * peak) / float(peak)
        return noisy

data_augmentation = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomZoom(0.1, fill_mode='nearest'),
    RandomRotation(0.2, fill_mode='nearest'),
    RandomContrast(0.3),
    NoiseLayer(noise_type='poisson')])

images = [pic1, pic2]
images = [cv2.resize(img, (800, 800)) for img in images]
print(images[0].shape, images[1].shape)
images = np.stack(images, axis=0)/255.
print(images.shape)

for i in range(9):
  augmented_images = data_augmentation(images)
  noisy_images = tf.clip_by_value(augmented_images, 0., 1.)
  for j in range(noisy_images.shape[0]):
    plt.figure()
    ax = plt.subplot(noisy_images.shape[0], noisy_images.shape[0], j + 1)
    plt.imshow(noisy_images[j])
    plt.axis("off")

In [ ]:
## Part 3
# Model architechture
# Keras to create baseline CNN model

In [ ]:
## Part 4
# Evaluation and testing